In [1]:
%load_ext micropython_magic


Current ip.InteractiveTB.mode='Context'
New ip.InteractiveTB.mode='Minimal'


In [14]:
# %%micropython
import os, json
import uhashlib, ubinascii  # type: ignore


def listdir(path=".", sub=False, JSON=True, hash=False):

    # Lists the file information of a folder
    li = []
    di = {}
    if path == ".":  # Get current folder name
        path = os.getcwd()
    files = os.listdir(path)
    for file in files:
        # get size of each file
        info = {"path": path, "Name": file, "Size": 0}
        if path[-1] == "/":
            full = "%s%s" % (path, file)
        else:
            full = "%s/%s" % (path, file)
        # print("os.stat({})".format( full))
        subdir = []
        try:
            stat = os.stat(full)
            if stat[0] & 0x4000:  # stat.S_IFDIR
                info["type"] = "dir"
                # recurse folder(s)
                if sub == True:
                    # print("Folder :{}".format( full))
                    subdir = listdir(path=full, sub=True, JSON=False, hash=hash)
            else:
                info["size"] = stat[6]
                info["type"] = "file"
                if hash:
                    with open(full, "rb") as f:
                        h = uhashlib.sha256(f.read())
                        info["hash"] = ubinascii.hexlify(h.digest())
        except OSError as e:
            # print("error:{} processing file:{}".format(e, full))
            info["error"] = e.args[0]
            info["type"] = "OSError"
        info["fullname"] = full
        di[full] = info
        li.append(info)
        # recurse folder(s)
        if sub == True:
            di[full] = subdir
            li = li + subdir  # type: ignore
    if JSON == True:
        # return json.dumps(li)
        return json.dumps(di)
    else:
        # return li
        return di


# listdir()

ModuleNotFoundError: No module named 'uhashlib'

In [12]:
# %%micropython
l = listdir(path=".", sub=True, JSON=True, gethash=True)
print(l)

TypeError: listdir() got an unexpected keyword argument 'gethash'

In [5]:

!mpremote run "tests/scripts/listdir_json.py" exec "print(listdir(sub=True, hash=True, JSON=True))"


[{"Size": 0, "type": "dir", "fullname": "/elsewhere", "path": "/", "Name": "elsewhere"}, {"Size": 0, "type": "dir", "fullname": "/folder_1", "path": "/", "Name": "folder_1"}, {"size": 19, "Size": 0, "path": "/folder_1", "Name": "bar.txt", "fullname": "/folder_1/bar.txt", "type": "file", "hash": "54388de1ef64b02b8272b0462beba78957c696a7f72fbbb1db917ae29d9b981b"}, {"Size": 0, "type": "dir", "fullname": "/folder_1/folder_2", "path": "/folder_1", "Name": "folder_2"}, {"size": 14, "Size": 0, "path": "/folder_1/folder_2", "Name": "foo.txt", "fullname": "/folder_1/folder_2/foo.txt", "type": "file", "hash": "cfce4e2952591e79a0dea1654a92dba4f099d348ab7c176bcd052d69b8929770"}, {"size": 19, "Size": 0, "path": "/folder_1", "Name": "foo.txt", "fullname": "/folder_1/foo.txt", "type": "file", "hash": "54388de1ef64b02b8272b0462beba78957c696a7f72fbbb1db917ae29d9b981b"}, {"Size": 0, "type": "dir", "fullname": "/lib", "path": "/", "Name": "lib"}, {"size": 202, "Size": 0, "path": "/lib", "Name": "__future

In [8]:
import subprocess, json

r = subprocess.run(
    [
        "mpremote",
        "run",
        "tests/scripts/listdir_json.py",
        "exec",
        "print(listdir('/',sub=True, hash=True,JSON=True))",
    ],
    capture_output=True,
)
r.stdout.decode("utf-8")
file_info = json.loads(r.stdout.decode("utf-8"))